In [ ]:
%load_ext autoreload
%autoreload 2

#%config IPCompleter.greedy=True

# Rotation curves

First, imports:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
from astropy import units
from pathlib import Path
import os
import astropy.units as u
from astropy.constants import G

import snapshot_obj 
import curve_fit
import dataset_compute

import importlib

In [ ]:
importlib.reload(snapshot_obj)
importlib.reload(curve_fit)
importlib.reload(dataset_compute)

## Get data

In [ ]:
# Which halo:
gn = 2; sgn = 1

# Get LCDM data:
LCDM = snapshot_obj.Snapshot("V1_LR_fix",127)

In [ ]:
# Read halo:
fnum = LCDM.file_of_halo(gn,sgn)
SGNs = LCDM.get_subhalos("SubGroupNumber",fnums=[fnum])
GNs = LCDM.get_subhalos("GroupNumber",fnums=[fnum])

vmax = LCDM.get_subhalos("Vmax",fnums=[fnum]) / 100000 
                                                    # cm/s to km/s 
rmax = LCDM.get_subhalos("VmaxRadius",fnums=[fnum]) \
            * u.cm.to(u.kpc)

# Select halo:
halo_mask = np.logical_and(SGNs == sgn,GNs == gn)
vmax = vmax[halo_mask]
rmax = rmax[halo_mask]

In [ ]:
# V1kpc:
SGNs = LCDM.get_subhalos("SubGroupNumber")
GNs = LCDM.get_subhalos( "GroupNumber")
halo_mask = np.logical_and(SGNs == sgn,GNs == gn)
#v1kpc = LCDM.get_subhalos("V1kpc") 
#v1kpc = v1kpc[halo_mask]

In [ ]:
r = {}
v_circ = {}
pts = {'gas' : [0], 'dm' : [1], 'stars' : [4], 'BHs' : [5], 'all' : [0,1,4,5]}
# Read particles:
for key,pt in pts.items():
    r_pt, v_circ_pt = dataset_compute.compute_rotation_curve(\
            LCDM, gn, sgn, part_type=pt)
    r[key] = r_pt * u.cm.to(u.kpc)
    v_circ[key] = v_circ_pt * u.cm.to(u.km) 


In [ ]:
fig, axes = plt.subplots()

# Set axes:
#axes.set_xscale('log')
#axes.set_yscale('log')
#axes.set_xlim(0,50)
#axes.set_ylim(y_down,y_up)

# Set labels:
axes.set_xlabel('$r[\mathrm{kpc}]$', fontsize=16)
axes.set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$', fontsize=16)

# Galaxy text:
axes.set_title('Rotation curve: GN={}, SGN={}'.format(gn,sgn))

for key in pts.keys():
    axes.plot(r[key], v_circ[key], label=key)
    
axes.axhline(vmax,linestyle='dashed')
axes.axvline(rmax,linestyle='dashed')
#axes.axhline(v1kpc,linestyle='dashed',c='black')
axes.axvline(1,linestyle='dashed',c='black')
    
plt.legend()
plt.tight_layout()


# Save figure:
#
#filename = 'cumulDistByVmax_evol_{}.png'.format(galaxy)
#
#home = os.path.dirname(snapshot_obj.__file__)
#path = os.path.join(home,"Figures")
#
## If the directory does not exist, create it
#if not os.path.exists(path):
#    os.makedirs(path)
#plt.savefig(os.path.join(path,filename), dpi=200)